In [1]:
import os
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pysentiment2 as ps
import string

# 將sentence轉為token

In [2]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for file in dir_path:
        if "_company.csv" in file:
            datapath.append(os.path.join(path, dirname, file))
print(len(datapath))
datapath[:5]

490


['C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\3M Co\\3M Co._company.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\A.O. Smith Corp\\A.O. Smith Corp._company.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\Abbott Laboratories\\Abbott Laboratories_company.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\AbbVie\\AbbVie_company.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\Abiomed\\Abiomed_company.csv']

In [3]:
for i in range(len(datapath)):
    print(i)
    path = datapath[i]
    data = pd.read_csv(path)


    data.drop_duplicates(subset=['headline'], inplace = True, ignore_index = True)#刪除重複sample
    data.drop(columns = ["link"], inplace = True)#刪除連結
    if 'time' in data.columns:
        data.drop(columns = ["time"], inplace = True)#後來抓得有些有時間


    data["head_token"] = None
    data["text_token"] = None


    stop_words = stopwords.words('english')#要刪掉的word
    exclude = set(string.punctuation)#要刪掉的標點符號
    for i in range(0, data.shape[0]):
        '''處理 Head'''
        #小寫
        text = data.iloc[i, 0].lower()
        # 移除標點符號
        text = ''.join(ch for ch in text if ch not in exclude)
        # token 句子
        tokens = word_tokenize(text)
        # 移除 stopwords
        tokens = [w for w in tokens if not w in stop_words]
        # 將 token 連成 str, 使用 , 連結
        data.iloc[i, -2] = ','.join(x for x in tokens)

        '''處理 text'''
        #小寫
        text = data.iloc[i, 2].lower()
        # 移除標點符號
        text = ''.join(ch for ch in text if ch not in exclude)
        # token 句子 
        tokens = word_tokenize(text)
        # 移除 stopwords
        tokens = [w for w in tokens if not w in stop_words]
        # 將 token 連成 str, 使用 , 連結
        data.iloc[i, -1] = ','.join(x for x in tokens)


    path = path.replace(".csv", "_b.csv")
    data.to_csv(path, index = False)#save


data.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,headline,date,text,head_token,text_token
0,"When Activists Enter the Kitchen, the CFOs Fee...",2017-06-12,Investors cheered last month when Whole Foods ...,"activists,enter,kitchen,cfos,feel,heat","investors,cheered,last,month,whole,foods,marke..."
1,The Reason Investors Love Spinoffs: Juicier Re...,2017-10-10,"Among corporate executives, spinoffs of divisi...","reason,investors,love,spinoffs,juicier,returns","among,corporate,executives,spinoffs,divisions,..."
2,Eli Lilly Exploring Spinoff of Elanco Animal-H...,2017-10-24,Eli LillyLLY -0.79%& Co. said Tuesday it could...,"eli,lilly,exploring,spinoff,elanco,animalhealt...","eli,lillylly,079,co,said,tuesday,could,spin,se..."
3,"Buybacks, Pension Contributions and ...",2018-01-17,Some of the biggest U.S. companies are promisi...,"buybacks,pension,contributions,savings,compani...","biggest,us,companies,promising,significant,ann..."
4,The Strong Case for Health-Care Spinoffs,2018-01-25,"When everyone is buying, often the best strate...","strong,case,healthcare,spinoffs","everyone,buying,often,best,strategy,sellerdeal..."


# 計算各情緒字的個數

In [4]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for file in dir_path:
        if ("_b.csv" in file):
            datapath.append(os.path.join(path, dirname, file))
print(len(datapath))
datapath[:5]

490


['C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\3M Co\\3M Co._company_b.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\A.O. Smith Corp\\A.O. Smith Corp._company_b.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\Abbott Laboratories\\Abbott Laboratories_company_b.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\AbbVie\\AbbVie_company_b.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\Abiomed\\Abiomed_company_b.csv']

In [5]:
for i in range(len(datapath)):
    print(i)
    path = datapath[i]
    b_data = pd.read_csv(path)


    b_data["head_pos"]      = 0
    b_data["head_neg"]      = 0
    b_data["head_neutral"]  = 0
    b_data["head_total"]    = 0
    b_data["text_pos"]      = 0
    b_data["text_neg"]      = 0
    b_data["text_neutral"]  = 0
    b_data["text_total"]    = 0


    hiv4 = ps.HIV4()
    for i in range(0, b_data.shape[0]):
        pos = 0
        neg = 0
        neutral = 0
        for word in b_data["head_token"][i].split(","):#用","把字切割成list
            tokens = hiv4.tokenize(word)#先把字放入hiv4.tokenize才能計算分數
            score = hiv4.get_score(tokens)#計算分數
            if(score["Polarity"] > 0):#正面
                pos = pos + 1
            elif (score["Polarity"] < 0) :#負面
                neg = neg + 1
            else:#中性
                neutral = neutral + 1
        b_data.iloc[i, -8] = pos
        b_data.iloc[i, -7] = neg
        b_data.iloc[i, -6] = neutral
        b_data.iloc[i, -5] = len(b_data["head_token"][i].split(","))#用","把自切割成list後計算長度-->總字數

        pos = 0
        neg = 0
        neutral = 0      
        for word in b_data["text_token"][i].split(","):#用","把字切割成list
            tokens = hiv4.tokenize(word)#先把字放入hiv4.tokenize才能計算分數
            score = hiv4.get_score(tokens)#計算分數
            if(score["Polarity"] > 0):#正面
                pos = pos + 1
            elif (score["Polarity"] < 0) :#負面
                neg = neg + 1
            else:#中性
                neutral = neutral + 1
        b_data.iloc[i, -4] = pos
        b_data.iloc[i, -3] = neg
        b_data.iloc[i, -2] = neutral
        b_data.iloc[i, -1] = len(b_data["text_token"][i].split(","))#用","把自切割成list後計算長度-->總字數


    path = path.replace("_b.csv", "_c.csv")
    b_data.to_csv(path, index = False)#save


b_data.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,headline,date,text,head_token,text_token,head_pos,head_neg,head_neutral,head_total,text_pos,text_neg,text_neutral,text_total
0,"When Activists Enter the Kitchen, the CFOs Fee...",2017-06-12,Investors cheered last month when Whole Foods ...,"activists,enter,kitchen,cfos,feel,heat","investors,cheered,last,month,whole,foods,marke...",0,0,6,6,105,36,659,800
1,The Reason Investors Love Spinoffs: Juicier Re...,2017-10-10,"Among corporate executives, spinoffs of divisi...","reason,investors,love,spinoffs,juicier,returns","among,corporate,executives,spinoffs,divisions,...",2,0,4,6,76,32,480,588
2,Eli Lilly Exploring Spinoff of Elanco Animal-H...,2017-10-24,Eli LillyLLY -0.79%& Co. said Tuesday it could...,"eli,lilly,exploring,spinoff,elanco,animalhealt...","eli,lillylly,079,co,said,tuesday,could,spin,se...",0,0,7,7,29,9,264,302
3,"Buybacks, Pension Contributions and ...",2018-01-17,Some of the biggest U.S. companies are promisi...,"buybacks,pension,contributions,savings,compani...","biggest,us,companies,promising,significant,ann...",4,1,3,8,79,35,432,546
4,The Strong Case for Health-Care Spinoffs,2018-01-25,"When everyone is buying, often the best strate...","strong,case,healthcare,spinoffs","everyone,buying,often,best,strategy,sellerdeal...",0,0,4,4,40,10,218,268


# 計算ratio

In [6]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for file in dir_path:
        if ("_c.csv" in file):
            datapath.append(os.path.join(path, dirname, file))
print(len(datapath))
datapath[:5]

490


['C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\3M Co\\3M Co._company_c.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\A.O. Smith Corp\\A.O. Smith Corp._company_c.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\Abbott Laboratories\\Abbott Laboratories_company_c.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\AbbVie\\AbbVie_company_c.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\Abiomed\\Abiomed_company_c.csv']

In [7]:
for i in range(len(datapath)):
    print(i)
    path = datapath[i]
    c_data = pd.read_csv(path)


    aggregation_functions = {'head_pos': 'sum', 'head_neg': 'sum', 'head_neutral': "sum",\
                            'head_total': 'sum', 'text_pos': 'sum', 'text_neg': "sum",\
                            'text_neutral': 'sum', 'text_total': 'sum',}
    #按照日期groupby，並加總
    d_data = c_data.groupby(c_data['date']).aggregate(aggregation_functions)
    d_data.reset_index(inplace = True)


    d_data['head_Pos_ratio']     = None
    d_data['head_Neg_ratio']     = None
    d_data['head_Netural_ratio'] = None
    d_data['head_Polarity']      = None
    d_data['head_Subjectivity']  = None
    d_data['text_Pos_ratio']     = None
    d_data['text_Neg_ratio']     = None
    d_data['text_Netural_ratio'] = None
    d_data['text_Polarity']      = None
    d_data['text_Subjectivity']  = None


    for w in range(0, d_data.shape[0]):
        if(d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w]!=0):#分母不可為0
            d_data.iloc[w, 9] = d_data["head_pos"].iloc[w]\
            /(d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w])#head_Pos_ratio

            d_data.iloc[w, 10] = d_data["head_neg"].iloc[w]\
            /(d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w])#head_Neg_ratio

            d_data.iloc[w, 12] = (d_data["head_pos"].iloc[w]-d_data["head_neg"].iloc[w])\
            /(d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w])#head_Polarity
        else:
            d_data.iloc[w, 9] = 0 #head_Pos_ratio
            d_data.iloc[w, 10] = 0 #head_Neg_ratio        
            d_data.iloc[w, 12] = 0 #head_Polarity

        if(d_data["head_total"].iloc[w]!=0):#分母不可為0
            d_data.iloc[w, 11] = d_data["head_neutral"].iloc[w]/d_data["head_total"].iloc[w]#head_Netural_ratio

            d_data.iloc[w, 13] = (d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w])/ \
            (d_data["head_total"].iloc[w])#head_Subjectivity
        else:
            d_data.iloc[w, 11] = 0 #head_Netural_ratio
            d_data.iloc[w, 13] = 0 #head_Subjectivity


        if(d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w]!=0):#分母不可為0
            d_data.iloc[w, 14] = d_data["text_pos"].iloc[w]\
            /(d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w])#text_Pos_ratio

            d_data.iloc[w, 15] = d_data["text_neg"].iloc[w]\
            /(d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w])#text_Neg_ratio

            d_data.iloc[w, 17] = (d_data["text_pos"].iloc[w]-d_data["text_neg"].iloc[w])\
            /(d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w])#text_Polarity     
        else:
            d_data.iloc[w, 14] = 0 #text_Pos_ratio
            d_data.iloc[w, 15] = 0 #text_Neg_ratio
            d_data.iloc[w, 17] = 0 #text_Polarity

        if(d_data["text_total"].iloc[w] != 0):#分母不可為0
            d_data.iloc[w, 16] =  d_data["text_neutral"].iloc[w]/d_data["text_total"].iloc[w]#text_Netural_ratio
            d_data.iloc[w, 18] = (d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w])\
            /d_data["text_total"].iloc[w]#text_Subjectivity
        else:
            d_data.iloc[w, 16] = 0 #text_Netural_ratio
            d_data.iloc[w, 18] = 0 #text_Subjectivity


    path = path.replace("_c.csv", "_d.csv")
    d_data.to_csv(path, index = False)#save


d_data.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,date,head_pos,head_neg,head_neutral,head_total,text_pos,text_neg,text_neutral,text_total,head_Pos_ratio,head_Neg_ratio,head_Netural_ratio,head_Polarity,head_Subjectivity,text_Pos_ratio,text_Neg_ratio,text_Netural_ratio,text_Polarity,text_Subjectivity
0,2017-06-12,0,0,6,6,105,36,659,800,0,0,1.0,0,0.0,0.744681,0.255319,0.82375,0.489362,0.17625
1,2017-10-10,2,0,4,6,76,32,480,588,1.0,0.0,0.666667,1.0,0.333333,0.703704,0.296296,0.816327,0.407407,0.183673
2,2017-10-24,0,0,7,7,29,9,264,302,0,0,1.0,0,0.0,0.763158,0.236842,0.874172,0.526316,0.125828
3,2018-01-17,4,1,3,8,79,35,432,546,0.8,0.2,0.375,0.6,0.625,0.692982,0.307018,0.791209,0.385965,0.208791
4,2018-01-25,0,0,4,4,40,10,218,268,0,0,1.0,0,0.0,0.8,0.2,0.813433,0.6,0.186567


# 透過standarization計算ratio

In [8]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for file in dir_path:
        if ("_d.csv" in file):
            datapath.append(os.path.join(path, dirname, file))
print(len(datapath))
datapath[:5]

490


['C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\3M Co\\3M Co._company_d.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\A.O. Smith Corp\\A.O. Smith Corp._company_d.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\Abbott Laboratories\\Abbott Laboratories_company_d.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\AbbVie\\AbbVie_company_d.csv',
 'C:\\Users\\i7-870\\Documents\\Python\\news\\WSJ\\abbrev_news\\Abiomed\\Abiomed_company_d.csv']

In [9]:
for i in range(len(datapath)):
    print(i)
    path = datapath[i]
    e_data = pd.read_csv(path)


    # standarization = (X - mean) / std
    e_data['head_sPos_ratio']     =\
    (e_data["head_Pos_ratio"]-e_data["head_Pos_ratio"].mean())/e_data["head_Pos_ratio"].std()

    e_data['head_sNeg_ratio']     =\
    (e_data["head_Neg_ratio"]-e_data["head_Neg_ratio"].mean())/e_data["head_Neg_ratio"].std()

    e_data['head_sNetural_ratio'] =\
    (e_data["head_Netural_ratio"]-e_data["head_Netural_ratio"].mean())/e_data["head_Netural_ratio"].std()

    e_data['head_sPolarity']      =\
    (e_data["head_Polarity"]-e_data["head_Polarity"].mean())/e_data["head_Polarity"].std()

    e_data['head_sSubjectivity']  =\
    (e_data["head_Subjectivity"]-e_data["head_Subjectivity"].mean())/e_data["head_Subjectivity"].std()

    e_data['text_sPos_ratio']     =\
    (e_data["text_Pos_ratio"]-e_data["text_Pos_ratio"].mean())/e_data["text_Pos_ratio"].std()

    e_data['text_sNeg_ratio']     =\
    (e_data["text_Neg_ratio"]-e_data["text_Neg_ratio"].mean())/e_data["text_Neg_ratio"].std()

    e_data['text_sNetural_ratio'] =\
    (e_data["text_Netural_ratio"]-e_data["text_Netural_ratio"].mean())/e_data["text_Netural_ratio"].std()

    e_data['text_sPolarity']      =\
    (e_data["text_Polarity"]-e_data["text_Polarity"].mean())/e_data["text_Polarity"].std()

    e_data['text_sSubjectivity']  =\
    (e_data["text_Subjectivity"]-e_data["text_Subjectivity"].mean())/e_data["text_Subjectivity"].std()
    
    
    #做standarization時，當std=0在分母，結果會是nan，須補0
    e_data.fillna(0, inplace = True)

    
    path = path.replace("_d.csv", "_e.csv")
    e_data.to_csv(path, index = False)#save

e_data.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,date,head_pos,head_neg,head_neutral,head_total,text_pos,text_neg,text_neutral,text_total,head_Pos_ratio,...,head_sPos_ratio,head_sNeg_ratio,head_sNetural_ratio,head_sPolarity,head_sSubjectivity,text_sPos_ratio,text_sNeg_ratio,text_sNetural_ratio,text_sPolarity,text_sSubjectivity
0,2017-06-12,0,0,6,6,105,36,659,800,0.0,...,-1.353744,-0.609495,1.084393,-0.548367,-1.084393,0.038554,-0.038554,-0.173023,0.038554,0.173023
1,2017-10-10,2,0,4,6,76,32,480,588,1.0,...,0.880591,-0.609495,-0.529482,0.864094,0.529482,-0.330797,0.330797,-0.396503,-0.330797,0.396503
2,2017-10-24,0,0,7,7,29,9,264,302,0.0,...,-1.353744,-0.609495,1.084393,-0.548367,-1.084393,0.205098,-0.205098,1.344910,0.205098,-1.344910
3,2018-01-17,4,1,3,8,79,35,432,546,0.8,...,0.433724,-0.049418,-1.941624,0.299109,1.941624,-0.427434,0.427434,-1.152659,-0.427434,1.152659
4,2018-01-25,0,0,4,4,40,10,218,268,0.0,...,-1.353744,-0.609495,1.084393,-0.548367,-1.084393,0.537178,-0.537178,-0.483616,0.537178,0.483616
